In [2]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from keras.layers import GlobalAveragePooling1D, Dense, Input
import transformers
from transformers import BertTokenizer, TFBertModel
from sklearn.model_selection import train_test_split
import os

2024-02-14 16:07:14.752826: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-14 16:07:14.752923: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-14 16:07:14.886121: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
train = pd.read_csv('/kaggle/input/tweet-diaster/train.csv', usecols=[3,4])

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"
gpu = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

In [5]:
train.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [7]:
def prepro(t):
    t = t.lower()
    t = re.sub(r"[^a-zA-Z\s]","",t)
    t = re.sub(r"won\'t", " will not", t)
    t = re.sub(r"won\'t've", " will not have", t)
    t = re.sub(r"can\'t", " can not", t)
    t = re.sub(r"don\'t", " do not", t)
    t = re.sub(r"can\'t've", " can not have", t)
    t = re.sub(r"ma\'am", " madam", t)
    t = re.sub(r"let\'s", " let us", t)
    t = re.sub(r"ain\'t", " am not", t)
    t = re.sub(r"shan\'t", " shall not", t)
    t = re.sub(r"sha\n't", " shall not", t)
    t = re.sub(r"o\'clock", " of the clock", t)
    t = re.sub(r"y\'all", " you all", t)
    t = re.sub(r"n\'t", " not", t)
    t = re.sub(r"n\'t've", " not have", t)
    t = re.sub(r"\'re", " are", t)
    t = re.sub(r"\'s", " is", t)
    t = re.sub(r"\'d", " would", t)
    t = re.sub(r"\'d've", " would have", t)
    t = re.sub(r"\'ll", " will", t)
    t = re.sub(r"\'ll've", " will have", t)
    t = re.sub(r"\'t", " not", t)
    t = re.sub(r"\'ve", " have", t)
    t = re.sub(r"\'m", " am", t)
    t = re.sub(r"\'re", " are", t)
    t = " ".join(re.findall(r"[^\W\d_]+|\d+",t))
    t = re.sub("\s+"," ", t)

    return t

In [8]:
train["text"] = train["text"].apply(prepro)

In [9]:
train.head()["text"][0]

'our deeds are the reason of this earthquake may allah forgive us all'

In [10]:
train.shape

(7613, 2)

In [11]:
Xtra, Xtes, Ytra, Ytes = train_test_split(train["text"], train["target"],test_size=0.15)

In [12]:
Xtra = tokenizer(
    Xtra.tolist(),
    max_length=128,
    truncation=True,
    padding='max_length',
    add_special_tokens=True,
    return_tensors='np'
)
Ytra = Ytra.values

In [13]:
Xtra["input_ids"].shape

(6471, 128)

In [14]:
Xtra["attention_mask"].shape

(6471, 128)

In [15]:
Ytra.shape

(6471,)

In [22]:
Xtes = tokenizer(
    Xtes.tolist(),
    max_length=128,
    truncation=True,
    padding='max_length',
    add_special_tokens=True,
    return_tensors='np'
)
Ytes = Ytes.values

In [31]:
model_ids  = Input(shape=(128, ), dtype = tf.int32)
model_mask = Input(shape=(128, ), dtype = tf.int32)

bert_model = TFBertModel.from_pretrained('bert-base-uncased')
x = bert_model(input_ids = model_ids, attention_mask = model_mask)
x = tf.keras.layers.GlobalAveragePooling1D()(x.last_hidden_state)
x = tf.keras.layers.Dense(512, activation='relu')(x)
outputs = Dense(2)(x)

model = tf.keras.Model(inputs = [model_ids, model_mask], outputs = outputs)

model.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = "mse",
    metrics=["accuracy"])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [33]:
model.fit((np.array(Xtra['input_ids']),
           np.array(Xtra['attention_mask'])),
    Ytra.ravel(), 
    epochs = 20,
    shuffle=True,                    
    batch_size = 16,
    validation_split=0.2 )

Epoch 1/20
324/324 [==============================] - 90s 278ms/step - loss: 0.2503 - accuracy: 0.5029 - val_loss: 0.2453 - val_accuracy: 0.4216
Epoch 2/20
324/324 [==============================] - 90s 278ms/step - loss: 0.2515 - accuracy: 0.5041 - val_loss: 0.2445 - val_accuracy: 0.5784
Epoch 3/20
324/324 [==============================] - 90s 277ms/step - loss: 0.2501 - accuracy: 0.5095 - val_loss: 0.2519 - val_accuracy: 0.5784
Epoch 4/20
324/324 [==============================] - 90s 277ms/step - loss: 0.2510 - accuracy: 0.4975 - val_loss: 0.2439 - val_accuracy: 0.4216
Epoch 5/20
324/324 [==============================] - 90s 277ms/step - loss: 0.2499 - accuracy: 0.4921 - val_loss: 0.2467 - val_accuracy: 0.4216
Epoch 6/20
324/324 [==============================] - 90s 278ms/step - loss: 0.2481 - accuracy: 0.4973 - val_loss: 0.2441 - val_accuracy: 0.4216
Epoch 7/20
324/324 [==============================] - 90s 278ms/step - loss: 0.2481 - accuracy: 0.4983 - val_loss: 0.2446 - val_ac

In [35]:
model.evaluate((np.array(Xtes['input_ids']),
           np.array(Xtes['attention_mask'])),
    Ytes.ravel())

36/36 [==============================] - 6s 162ms/step - loss: 0.2427 - accuracy: 0.5902


[0.24266394972801208, 0.5901926159858704]